In [1]:
%load_ext autoreload
%autoreload 2

from domino.slices import collect_settings
from domino.slices.celeba import CelebASliceBuilder
from domino.evaluate import run_sdm, score_sdms
from domino.train import train_settings
import numpy as np
import terra
from domino.emb.clip import generate_phrases, CELEBA_PHRASE_TEMPLATES


/home/sabri/code/meerkat/meerkat/nn/__init__.py:7: ExperimentalWarning: The `meerkat.nn` module is experimental and has limited test coverage. Proceed with caution.
  warnings.warn(


In [19]:
dp =generate_phrases.out().load()

In [20]:
dp.lz[dp["loss"].argsort()[:100]]

,loss (NumpyArrayColumn),output_phrase (PandasSeriesColumn),index (PandasSeriesColumn)
0,17.355467,a photo of a person holding a bouquet of flowers.,14839
1,18.182400,a photo of a person with rheumatoid arthritis.,2549
2,18.708553,a photo of a person with a congenital heart defect.,22890
3,18.810265,a photo of a person with an umbilical cord.,11026
4,18.846235,a photo of a person wearing a bulletproof vest.,12324
...,...,...,...
95,25.594769,a photo of a person with a tuft of hair.,19320
96,25.610512,a photo of a person holding a goblet of water.,40927
97,25.741375,a photo of a person in a kimono.,15473
98,25.749462,a photo of a person wearing a blouse and skirt.,40959


In [16]:
from domino.emb.clip import get_wiki_words

dp = get_wiki_words(top_k=25_001, eng_only=True)

task: get_wiki_words, run_id=16831


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
from domino.emb.clip import generate_phrases, CELEBA_PHRASE_TEMPLATES
phrase_dp = generate_phrases(dp.load()[:10], templates=CELEBA_PHRASE_TEMPLATES, num_candidates=5000)

task: generate_phrases, run_id=16829


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
collect_settings.out().load()

,alpha (NumpyArrayColumn),build_setting_kwargs (ListColumn),dataset (PandasSeriesColumn),slice_category (PandasSeriesColumn),slice_names (ListColumn),target_name (PandasSeriesColumn),index (PandasSeriesColumn),setting_id (PandasSeriesColumn)
0,0.0,"{'n': 30000, 'correlate': 'blond_hair', 'target': 'bangs', 'corr': 0.0}",celeba,correlation,"['bangs=0_blond_hair=1', 'bangs=1_blond_hair=0']",bangs,0,ff44e2cc-9087-4dec-8093-0d4e46f5fa6f
1,0.2,"{'n': 30000, 'correlate': 'blond_hair', 'target': 'bangs', 'corr': 0.2}",celeba,correlation,"['bangs=0_blond_hair=1', 'bangs=1_blond_hair=0']",bangs,1,c48ae98b-77ae-426c-a240-4a84a7808975
2,0.4,"{'n': 30000, 'correlate': 'blond_hair', 'target': 'bangs', 'corr': 0.4}",celeba,correlation,"['bangs=0_blond_hair=1', 'bangs=1_blond_hair=0']",bangs,2,6131f7d1-64d4-408a-a462-c4f725800af6
3,0.6,"{'n': 30000, 'correlate': 'blond_hair', 'target': 'bangs', 'corr': 0.6000000000000001}",celeba,correlation,"['bangs=0_blond_hair=1', 'bangs=1_blond_hair=0']",bangs,3,631eadef-9fcd-4328-b1bc-b6ce2fff65c4
4,0.8,"{'n': 30000, 'correlate': 'blond_hair', 'target': 'bangs', 'corr': 0.8}",celeba,correlation,"['bangs=0_blond_hair=1', 'bangs=1_blond_hair=0']",bangs,4,4800699b-9ff2-442d-a3f8-221b71842c17
...,...,...,...,...,...,...,...,...
175,0.0,"{'n': 30000, 'correlate': 'no_beard', 'target': 'young', 'corr': 0.0}",celeba,correlation,"['young=0_no_beard=1', 'young=1_no_beard=0']",young,175,9bf8fab2-26d1-452f-ac9b-f4311255b56d
176,0.2,"{'n': 30000, 'correlate': 'no_beard', 'target': 'young', 'corr': 0.2}",celeba,correlation,"['young=0_no_beard=1', 'young=1_no_beard=0']",young,176,6087f315-203e-434d-8732-91619b6dc89d
177,0.4,"{'n': 30000, 'correlate': 'no_beard', 'target': 'young', 'corr': 0.4}",celeba,correlation,"['young=0_no_beard=1', 'young=1_no_beard=0']",young,177,710acc10-9285-4753-9ece-bdc9a5f31ca3
178,0.6,"{'n': 30000, 'correlate': 'no_beard', 'target': 'young', 'corr': 0.6000000000000001}",celeba,correlation,"['young=0_no_beard=1', 'young=1_no_beard=0']",young,178,ddbd783e-5d9d-45e1-a257-566b3b9445b9


In [2]:
from domino.emb.clip import get_wiki_words
#dp = get_wiki_words(top_k=25_000, eng_only=True).load()
word_dp = get_wiki_words.out().load()

In [52]:
from domino.emb.clip import generate_phrases, CELEBA_PHRASE_TEMPLATES
phrase_dp = generate_phrases(word_dp[:1000], templates=CELEBA_PHRASE_TEMPLATES, num_candidates=5000)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
phrase_dp.to_pandas().sort_values("loss", ascending=True)[:10]

,loss,output_phrase,index
2280,24.308901,a photo of a person with the same name.,2280
1398,24.415701,a photo of a person sitting next to him.,1398
1709,24.538908,a photo of a person standing next to him.,1709
1097,24.756279,a photo of a person wearing a baseball cap.,1097
731,25.882910,a photo of a person with a heart attack.,731
1039,26.439959,a photo of a person leaning against a wall.,1039
1434,26.695847,a photo of a person with a dead body.,1434
1906,26.843758,a photo of a person who is still alive.,1906
1070,28.207479,a photo of a person wearing a military uniform.,1070
272,28.332619,a photo of a person on the ground.,272


In [3]:
word_dp = generate_phrases.out().load()

In [7]:
dp = word_dp.to_pandas().sort_values("loss")[:10_000]

In [23]:
dp[dp["output_phrase"].str.contains("necklace")]

,loss,output_phrase,index
16961,26.456167,a photo of a person wearing a necklaces.,16961
39252,28.030666,a photo of a person wearing a sapphire necklace.,39252
49988,31.198801,a photo of a person with a necklaces.,49988
12815,37.864956,a photo of a person wearing necklaces.,12815
5676,39.922859,a photo of a person wearing a necklace.,5676
29685,40.586658,a photo of a person wearing a diamond necklace.,29685
19476,40.828266,a photo of a person wearing an emerald necklace.,19476
42287,41.489494,a photo of a person wearing a sapphire and nec...,42287
28598,42.364773,a photo of a person wearing an emerald green n...,28598
48166,43.971405,a photo of a person wearing a jeweled necklace.,48166


In [31]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np 
from tqdm import tqdm

gpt_model = GPT2LMHeadModel.from_pretrained('gpt2').to(0)
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

scores = []
for text in tqdm(candidate_sents["output_phrase"]):
    tokens_tensor = gpt_tokenizer.encode( text, add_special_tokens=False, return_tensors="pt").to(0) 
    loss=gpt_model(tokens_tensor, labels=tokens_tensor)[0]
    scores.append( 
        {"prob": np.exp(loss.cpu().detach().numpy()), "text": text}       
    )
dp = pd.DataFrame(scores)

100%|██████████| 1000/1000 [00:14<00:00, 68.04it/s]


In [32]:
dp.sort_values("prob")

,prob,text
439,56.578079,a photo of a person in the sea.
68,58.034519,a photo of a person with a guitar.
648,61.561375,a photo of a person walking through traffic.
487,61.679295,a photo of a person in another picture.
504,67.137917,a photo of a person with a debt.
...,...,...
376,1872.045166,a photo of a person constitution paper.
936,1957.345947,a photo of a person ferryeded.
293,2780.082520,a photo of a person cooper had from.
220,3535.363525,a photo of a person regional orized.


In [5]:
from domino.emb.clip import CELEBA_PHRASE_TEMPLATES
templates = CELEBA_PHRASE_TEMPLATES
words = word_dp["word"][:10]
input_phrases = [
    template.format(word) for word in words for template in templates
]
inputs = tokenizer(input_phrases, return_tensors="pt", padding=True).to(device)
input_ids = inputs["input_ids"]

outputs = model(**inputs) # shape=(num_sents, num_tokens_in_sent, size_vocab)

In [66]:
probs = torch.softmax(outputs.logits, dim=-1)

In [74]:
k = 2
top_k_out = probs.topk(k=k, dim=-1)
PAD_TOKEN_ID = 103

output_phrases = []
output_probs = []
for sent_idx in range(probs.shape[0]):
    mask_mask = input_ids[sent_idx] == PAD_TOKEN_ID
    token_ids = top_k_out.indices[sent_idx, mask_mask]
    token_probs = top_k_out.values[sent_idx, mask_mask]
    for local_idxs in product(np.arange(k), repeat=mask_mask.sum()):
        output_ids = torch.clone(input_ids[sent_idx])
        output_ids[mask_mask] = token_ids[torch.arange(mask_mask.sum()), local_idxs] 
        output_phrases.append(tokenizer.decode(output_ids, skip_special_tokens=True))
        output_probs.append(token_probs[torch.arange(mask_mask.sum()), local_idxs].mean().cpu().detach().numpy())



In [80]:
import pandas as pd
df = pd.DataFrame({"prob": output_probs, "phrase": output_phrases})
df.sort_values("prob", ascending=False)

,prob,phrase
0,nan,a photo of a person the.
454,0.49835655,a photo of a person dressed as a child.
442,0.49652216,a photo of a person as well.
455,0.48906392,a photo of a person dressed as a model.
475,0.3766427,a photo of a person dressed dressed as a person.
450,0.3659794,a photo of a person dressed as well.
441,nan,a photo of a person as.
401,0.37600285,a photo of a person sitting on it.
425,0.3701461,a photo of a person was placed on the wall.
429,0.36785626,a photo of a person was sitting on the wall.


In [ ]:
local_idxs

(1,)

In [49]:
from itertools import product
x[np.array(list(product(np.arange(k), repeat=4)))]

IndexError: too many indices for tensor of dimension 2

In [24]:
import hashlib

[autoreload of terra failed: Traceback (most recent call last):
  File "/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/common/envs/conda/envs/domino/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/common/envs/conda/envs/domino/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 981, in get_code
  File "<frozen importlib._bootstrap_external>", line 911, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_

In [31]:
hashlib.sha1("hello".encode('utf-8')).hexdigest()

'aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d'